In [1]:
import os
import re
from bs4 import BeautifulSoup
import requests
import json

In [2]:
dir = 'Articles'
articles = ['https://www.nytimes.com/interactive/2022/sports/olympics/skiing-millie-knight-paralympics-fear.html',
'https://www.nytimes.com/interactive/2022/04/30/us/tucker-carlson-tonight.html']
html_pages = []


for article in articles:
    response = requests.get(article)
    if response:
        html_pages.append(response.text)
    
    

In [3]:
def get_attribute_value(attribute):
    attributes = element.get(attribute)
    if type(attributes) == list:
        return ' '.join(attributes)
    else:
        return attributes
    
    
def analyse_transitions(components):
    transitions = []
    current = ''
    for c in components:
        if c != current:
            transitions.append(c)
            current = c
    return transitions
    

In [4]:
annotations = []

for i,html_page in enumerate(html_pages):
    print(articles[i])
    components = []

    soup = BeautifulSoup( html_page ,"lxml")
    body = soup.find('body' )

    for element in body.findChildren():

        attributes = ''
        for attr in element.attrs:
            attributes += attr + ' ' + ' '.join(element.attrs[attr])


        if element.name == 'p':

            attribute_value = get_attribute_value('class')
            if attribute_value == 'g-body':
                print(f'Body text:\n{element.text.strip()}')
                components.append('body_text')
            if attribute_value == 'g-fg':
                print(f'Scroll text:\t{element.text.strip()}\n')
                components.append('scroll_text')
            if re.search( 'g-cue' , str(attribute_value) ):
                print(f'Subtitle in video:\t{element.text}')  
                components.append('subtitle')


        if element.name == 'img':

            attribute_value = get_attribute_value('class')

            if re.search( r'g-overlay-play', str(attribute_value) ):
                print('Video\n')
                components.append('video')
        
        
    if len(components) == 0:
        
        scripts = soup.find_all('script')
        for s in scripts:
            if re.search( 'var props =' , str(s) ):
                lines = re.split( r'\n' , str(s) )
                json_str = ''
                for line in lines:
                    if re.search( 'var props =' , line):
                        json_str = re.sub( 'var props =' , '' , line ).strip()
                        index_slide = json_str.index('slides:') + len('slides:')
                        json_str = json_str[index_slide:]
                        json_str = re.sub(r'}$' , '' ,json_str.strip() ).strip()
                        json_str = re.sub( r'[\t\n]+' , '' , json_str )


                        index_slide = json_str.index('processedSlides:') 
                        json_str = json_str[:index_slide]
                        json_str = re.sub(',$', '' , json_str.strip() )


                if len(json_str)>0:
                    #print(json_str)
                    out = open('article.json','w',encoding='utf-8')
                    out.write(json_str)
                    out.close()
                    json_data = json.loads(json_str)
                    for item in json_data:
                        if 'video' in item:
                            components.append('video')

                        elif 'image' in item:
                            print(item['image'])
                            components.append('photograph')

                        elif 'text' in item:
                            print(item['text'])
                            components.append('text')

                
    annotations.append(components)






https://www.nytimes.com/interactive/2022/sports/olympics/skiing-millie-knight-paralympics-fear.html
Video

Subtitle in video:	It does seem a bit weird being visually impaired throwing

Subtitle in video:	myself down a mountain.

Subtitle in video:	I think that there must be some form of screw loose in my head.

Subtitle in video:	

Subtitle in video:	I never really knew until fairly recently, to be honest,

Subtitle in video:	that the way that I see is different to the way that

Subtitle in video:	everybody else sees.

Subtitle in video:	Just because I couldn’t see didn’t mean that I wouldn’t be

Subtitle in video:	going on the family ski trips.

Subtitle in video:	I went on one trip and, from the moment I

Subtitle in video:	drove up the mountain and saw so much snow, I just thought,

Subtitle in video:	“Yeah, this is the sport for me.”

Subtitle in video:	I have replicated my sights.

Subtitle in video:	I roughly have sort of 5 percent vision,

Subtitle in video:	and it’s just periph

In [5]:
for i,article in enumerate(annotations):
    print(articles[1])
    transitions = analyse_transitions(article)
    for t in transitions:
        print(t)

https://www.nytimes.com/interactive/2022/04/30/us/tucker-carlson-tonight.html
video
subtitle
body_text
video
body_text
scroll_text
body_text
video
subtitle
body_text
video
subtitle
video
https://www.nytimes.com/interactive/2022/04/30/us/tucker-carlson-tonight.html
video
photograph
text
video
photograph
video
photograph
video
text
photograph
video
text
photograph
video
text
video
photograph
text
photograph
video
text
photograph
video
text
photograph
video
text
photograph
video
text
video
text
video
text
photograph
video
text
photograph
video
text


In [6]:
parsed = json.loads(json_str)
print(json.dumps(parsed, indent=4))

[
    {
        "text": "Night after night, the host of the most-watched show in prime-time cable news uses a simple narrative to instill fear in his viewers: \u2018<span class=\"g-text they\">They</span>\u2019 want to control and then destroy \u2018<span class=\"g-text you\">you</span>.\u2019",
        "theme": "marquee",
        "video": "opengrid1_v22",
        "video_mobile": "opengrid1-mobile_v22",
        "video_position": "full",
        "video_loop": "true",
        "video_fallback": "https://static01.nyt.com/newsgraphics/2022/03/24/tucker-carlson-tonight/assets/opening-video-fallback.gif",
        "video_suppress_progress": "true",
        "video_suppress_mute_button": "true",
        "stepper": "false",
        "cta": "3"
    },
    {
        "text": "",
        "theme": "marquee",
        "video": "theyyou_v22",
        "video_mobile": "theyyou-mobile_v22",
        "video_position": "full",
        "video_loop": "false",
        "stepper": "false",
        "cta": "video"
   